In [22]:
import pandas as pd
import difflib
import ast
from collections import defaultdict
import re

In [7]:
df = pd.read_excel("table_1951.xlsx", names=["index", "lastname", "firstname", "org", "job", "loc", "line"])
df = df.iloc[:, 1:]

In [8]:
df

,lastname,firstname,org,job,loc,line
0,Abächerli-Delmonico,Jos,NaN,['professeur'],['r.Ste-Beuve 7'],"Abächerli-Delmonico Jos., professeur, r.Ste-Be..."
1,NaN,NaN,"Abattoirs, Malley",[],['Bureaud’inspection de la gare'],"Abattoirs, Malley. T. 24 8181. Bureaud’inspect..."
2,NaN,NaN,Abbaye de l',['Arc (cercle)'],['Montbenon 1.T. 22'],"Abbaye de l'Arc (cercle), Montbenon 1.T. 22 85..."
3,Abbet,Raymond,NaN,['cuisinier'],['Bereiéres 20'],"Abbet Raymond, cuisinier. Bereiéres 20."
4,Abbet,Stéphanie,NaN,[],['ch. de Pré-Fleuri 6'],"— Stéphanie, ch. de Pré-Fleuri 6."
...,...,...,...,...,...,...
53200,Zwolska,Jeanne,NaN,['dir. Clinique de Bois-Cerf'],['av. Ouchy 33'],"Zwolska Jeanne, dir. Clinique de Bois-Cerf, av..."
53201,Zynda,Ellen,NaN,['vendeuse'],['ch. de Mon-tolivet 14'],"Zynda Ellen, vendeuse, ch. de Mon-tolivet 14."
53202,Zysset,Marguerite,NaN,['secrét.'],['av. Maria-Belgia 1'],"Zysset Marguerite, secrét., av. Maria-Belgia 1."
53203,Zysset,Rita,NaN,[],['av. Maria-Belgia 1'],— Rita. av. Maria-Belgia 1.


In [9]:
STREET_CAT = ["Acacias", "Bains", "Roseneck", "Beau-Rivage", "Beauregard", "Maria-Belgia", "Belle-source", "François-Bocion", "Jean-Louis-de-Bons", "Brillancourt", "Bruyères", "Cart", "Closelet", "Cour", "Crêt", "Dapples", "Église anglaise", "Pré-fleuri", "Joliette", "Délices", "Épinettes", "Fleurettes", "Mont-Tendre", "Floréal", "Fontenailles", "Fontenay", "Fraisse", "Jordils", "Funiculaire", "Grammont", "Grancy", "Grande-rive", "Grasset", "Harpe", "Navigation", "Liseron", "Milan", "Mollendruz", "Mon-loisir", "Mont-d’or", "Montriond", "Mouettes", "Ouchy", "Auguste-Pidou", "Plaines", "Port", "Pré-Fleuri", "Rod", "Rond-Point", "Tilleuls", "Servan", "Suchet", "Voltaire", "Warney"
]

In [21]:
len(STREET_CAT)

54

In [10]:
def clean_street(word):
    ABREV = ['av.', 'r.', 'ch.', ' du ', ' des ', 'de la', 'avenue', 'rue', 'passage', 'aven', 'bd.', 'bdde', 'av ']

    if len(word) < 3:
        return ""
    
    word = word.replace(".", "")
    word = word.replace(",", "")

    word = ''.join([i for i in word if not i.isdigit()])

    for ab in ABREV:
        word = word.replace(ab, "")
    
    if len(word) > 0 and word[0] == ' ':
        word = word[1:]
    return word

In [11]:
final_streets_dict = defaultdict(list)
count = 0

for index, row in df.iterrows():
    streets_in_row = row['loc']
    row_street_list = ast.literal_eval(streets_in_row)
    for row_street in row_street_list:
        new_street = clean_street(row_street)

        result = difflib.get_close_matches(new_street, STREET_CAT, n=1, cutoff=0.65)
        if result:
            count += 1
            print(f"looking for {new_street} got {result[0]}")

            final_streets_dict[result[0]].append(index)

final_streets_dict
print(count)

looking for ch de Pré-Fleuri  got Pré-Fleuri
looking for chde Fontenay  got Fontenay
looking for Harpe  got Harpe
looking for d’Ouchy  got Ouchy
looking for r Voltaire  got Voltaire
looking for Harpe  got Harpe
looking for Montchoisi  got Mon-loisir
looking for avServan  got Servan
looking for chMollendruz  got Mollendruz
looking for chdes Bruyères  got Bruyères
looking for chde Belle-Source  got Belle-source
looking for ch de Beau-Rivage got Beau-Rivage
looking for ch deBeau-Rivage  got Beau-Rivage
looking for d’Ouchy  got Ouchy
looking for ch de Fontenay  got Fontenay
looking for Montriond  got Montriond
looking for Dapples  got Dapples
looking for Dapples  got Dapples
looking for Dapples  got Dapples
looking for Dapples  got Dapples
looking for Har^e  got Harpe
looking for Mon-Loisir  got Mon-loisir
looking for Seryan  got Servan
looking for rCrêtes  got Crêt
looking for Grancy  got Grancy
looking for avFloréal  got Floréal
looking for chSuchet  got Suchet
looking for Fleurîtes  got

In [12]:
rows_list = []

for street_key in final_streets_dict.keys():

    for index in final_streets_dict[street_key]:
        row = df.iloc[index]
        new_row = {
            'cat_loc': street_key,
            'lastname': row['lastname'],
            'firstname': row['firstname'],
            'org': row['org'],
            'job': row['job'],
            'loc': row['loc'],
            'line': row['line'],
        }
        rows_list.append(new_row)

df_index_street = pd.DataFrame(rows_list)

df_index_street

,cat_loc,lastname,firstname,org,job,loc,line
0,Pré-Fleuri,Abbet,Stéphanie,NaN,[],['ch. de Pré-Fleuri 6'],"— Stéphanie, ch. de Pré-Fleuri 6."
1,Pré-Fleuri,Amandmz-Dnpertnis,Victor,NaN,['ingénieurOFF'],['ch. Pré-Fleuri 2'],"Amandmz-Dnpertnis Victor, ingénieurOFF., ch. P..."
2,Pré-Fleuri,Berenger-Jalla,Alfred,NaN,[],['ch. Pré-Fleuri 4.T.'],"Berenger-Jalla Alfred, ch. Pré-Fleuri 4.T. 26 ..."
3,Pré-Fleuri,Bussy-Du,NaN,NaN,['ingénieur'],"['Pasquier Ernest', 'Dir des Forces de Joux', ...","Bussy-Du Pasquier Ernest, ingénieur,Dir des Fo..."
4,Pré-Fleuri,Capt,Philippe,NaN,[],['ch. de Pré-Fleuri 4'],"— Philippe, ch. de Pré-Fleuri 4."
...,...,...,...,...,...,...,...
6727,Brillancourt,Pagani,Emile,NaN,['chauffeur'],['ch. de Bril-lancourt 2'],"Pagani Emile, chauffeur, ch. de Bril-lancourt 2."
6728,Brillancourt,Pagani,Lucie,NaN,[],['ch. de Brillancourt 4'],"— Lucie, ch. de Brillancourt 4."
6729,Brillancourt,Rahm-Malherbe,Victor,NaN,['compt.'],['chem.Brillancourl 4'],"Rahm-Malherbe Victor, compt., chem.Brillancour..."
6730,Brillancourt,Rod-von Gunten,Mathilde,NaN,['Vve'],['ch. deBrillancourt 4'],"Rod-von Gunten Mathilde, Vve, ch. deBrillancou..."


In [13]:
df_index_street[df_index_street['cat_loc'] == 'Rond-Point']

,cat_loc,lastname,firstname,org,job,loc,line
4621,Rond-Point,Aerschmann,Arnold,NaN,"['empi, bur']",['av. duRond-Point 18.'],"Aerschmann Arnold, empi, bur., av. duRond-Poin..."
4622,Rond-Point,Aeschimann-Cnrchod,Alfred,NaN,['représentant'],['av. du Rond-Point 5.'],"Aeschimann-Cnrchod Alfred, représentant, av. d..."
4623,Rond-Point,Albiez,Georges,NaN,[],['av. du Rond-Point 15.'],"— Georges, av. du Rond-Point 15."
4624,Rond-Point,Albin-Michaud,Joseph,NaN,['concierge'],['avRont-Point 15.'],"Albin-Michaud Joseph, concierge, avRont-Point 15."
4625,Rond-Point,Allemann,Anne,NaN,['serveuse'],['Rd-Point 3.'],"—• Anne-Marie, serveuse, Rd-Point 3."
...,...,...,...,...,...,...,...
4888,Rond-Point,Willen-Dnpertnis,Henriette,NaN,['Vve'],['avendu Rond-Point 21.'],"Willen-Dnpertnis Henriette. Vve, avendu Rond-P..."
4889,Rond-Point,Wnthrich-Mages,Marcel,NaN,['relieur'],['av. duRond-Point 9.'],"Wnthrich-Mages Marcel, relieur, av. duRond-Poi..."
4890,Rond-Point,Zeller-Barblan,Claude,NaN,['relieur'],"['av. duRond-Point 7.', 'At. rue(Mercerie 9']","Zeller-Barblan Claude, relieur, av. duRond-Poi..."
4891,Rond-Point,Zimmermann,Maria,NaN,['sommelière'],['av. du Rond-Point 21'],"— Maria, sommelière. av. du Rond-Point 21."


In [14]:
df_index_street.to_csv("1951_index_street.csv", index=False)

In [15]:
df_index_street_clean = pd.read_csv("1951_index_street.csv")

# Street cleaning (we only keep relevant streets)

In [16]:
for index, row in df_index_street_clean.iterrows():
    streets_in_row = row['loc']
    #row_street_list = ast.literal_eval(streets_in_row)

    filtered_street = ""

    for row_street in streets_in_row:
        new_street = clean_street(row_street)
        result = difflib.get_close_matches(new_street, STREET_CAT, n=1, cutoff=0.65)

        if result:
            df_index_street_clean.loc[index, 'loc'] = row_street

In [17]:
df_index_street.to_csv("1951_index_street_final.csv", index=False)

# Street number extraction

In [18]:
df_index_street_final = pd.read_csv("1951_index_street_final.csv")

In [19]:
df_index_street_final

,cat_loc,lastname,firstname,org,job,loc,line
0,Pré-Fleuri,Abbet,Stéphanie,NaN,[],['ch. de Pré-Fleuri 6'],"— Stéphanie, ch. de Pré-Fleuri 6."
1,Pré-Fleuri,Amandmz-Dnpertnis,Victor,NaN,['ingénieurOFF'],['ch. Pré-Fleuri 2'],"Amandmz-Dnpertnis Victor, ingénieurOFF., ch. P..."
2,Pré-Fleuri,Berenger-Jalla,Alfred,NaN,[],['ch. Pré-Fleuri 4.T.'],"Berenger-Jalla Alfred, ch. Pré-Fleuri 4.T. 26 ..."
3,Pré-Fleuri,Bussy-Du,NaN,NaN,['ingénieur'],"['Pasquier Ernest', 'Dir des Forces de Joux', ...","Bussy-Du Pasquier Ernest, ingénieur,Dir des Fo..."
4,Pré-Fleuri,Capt,Philippe,NaN,[],['ch. de Pré-Fleuri 4'],"— Philippe, ch. de Pré-Fleuri 4."
...,...,...,...,...,...,...,...
6727,Brillancourt,Pagani,Emile,NaN,['chauffeur'],['ch. de Bril-lancourt 2'],"Pagani Emile, chauffeur, ch. de Bril-lancourt 2."
6728,Brillancourt,Pagani,Lucie,NaN,[],['ch. de Brillancourt 4'],"— Lucie, ch. de Brillancourt 4."
6729,Brillancourt,Rahm-Malherbe,Victor,NaN,['compt.'],['chem.Brillancourl 4'],"Rahm-Malherbe Victor, compt., chem.Brillancour..."
6730,Brillancourt,Rod-von Gunten,Mathilde,NaN,['Vve'],['ch. deBrillancourt 4'],"Rod-von Gunten Mathilde, Vve, ch. deBrillancou..."


In [20]:
street_number = []

max_iter = 0

for index, row in df_index_street_final.iterrows():
    street_in_row = row['loc']

    tel_sep = 'T'
    street_stripped = street_in_row.split(tel_sep, 1)[0] #to remove the telephone number from the street

    digits_group = re.findall('\d+', street_stripped)

    # digits_group should only have one number (ie. a list of one element) or nothing if there is no digits

    if len(digits_group) == 0:
        street_number.append(0) # street numbers cannot be 0 and from what I checked, I did not detect OCR failure
    elif len(digits_group) > 1:
        print("should not happen", digits_group, row)
        raise Exception
    else:
        street_number.append(int(digits_group[0]))

should not happen ['8', '39'] cat_loc                                               Fontenay
lastname                                       Audriaz-Mandrin
firstname                                              Léopold
org                                                        NaN
job                                               ['coiffeur']
loc                   ['ch.Fontenay 8', 'Mag. bdde Grancy 39']
line         Audriaz-Mandrin Léopold, coiffeur, ch.Fontenay...
Name: 40, dtype: object


Exception: 

In [ ]:
df_index_street_final.insert(len(df_index_street_final.columns) - 1, "street_num", street_number)

In [ ]:
df_index_street_final.to_csv('1951_index_street_final_best.csv')

In [ ]:
df_index_street_final

,cat_loc,lastname,firstname,org,job,loc,street_num,line
0,Pré-Fleuri,Abbet,Stéphanie,NaN,[],ch. de Pré-Fleuri 6,6,"— Stéphanie, ch. de Pré-Fleuri 6."
1,Pré-Fleuri,Amandmz-Dnpertnis,Victor,NaN,['ingénieurOFF'],ch. Pré-Fleuri 2,2,"Amandmz-Dnpertnis Victor, ingénieurOFF., ch. P..."
2,Pré-Fleuri,Berenger-Jalla,Alfred,NaN,[],ch. Pré-Fleuri 4.T.,4,"Berenger-Jalla Alfred, ch. Pré-Fleuri 4.T. 26 ..."
3,Pré-Fleuri,Bussy-Du,NaN,NaN,['ingénieur'],chemindu Pré-Fleuri 8,8,"Bussy-Du Pasquier Ernest, ingénieur,Dir des Fo..."
4,Pré-Fleuri,Capt,Philippe,NaN,[],ch. de Pré-Fleuri 4,4,"— Philippe, ch. de Pré-Fleuri 4."
...,...,...,...,...,...,...,...,...
6727,Brillancourt,Pagani,Emile,NaN,['chauffeur'],ch. de Bril-lancourt 2,2,"Pagani Emile, chauffeur, ch. de Bril-lancourt 2."
6728,Brillancourt,Pagani,Lucie,NaN,[],ch. de Brillancourt 4,4,"— Lucie, ch. de Brillancourt 4."
6729,Brillancourt,Rahm-Malherbe,Victor,NaN,['compt.'],chem.Brillancourl 4,4,"Rahm-Malherbe Victor, compt., chem.Brillancour..."
6730,Brillancourt,Rod-von Gunten,Mathilde,NaN,['Vve'],ch. deBrillancourt 4,4,"Rod-von Gunten Mathilde, Vve, ch. deBrillancou..."


# Occupation extraction

In [ ]:
df_index_street_final_best = pd.read_csv("1951_index_street_final_best.csv", index_col=0)

In [ ]:
df_index_street_final_best

,cat_loc,lastname,firstname,org,job,loc,street_num,line
0,Pré-Fleuri,Abbet,Stéphanie,NaN,[],ch. de Pré-Fleuri 6,6,"— Stéphanie, ch. de Pré-Fleuri 6."
1,Pré-Fleuri,Amandmz-Dnpertnis,Victor,NaN,['ingénieurOFF'],ch. Pré-Fleuri 2,2,"Amandmz-Dnpertnis Victor, ingénieurOFF., ch. P..."
2,Pré-Fleuri,Berenger-Jalla,Alfred,NaN,[],ch. Pré-Fleuri 4.T.,4,"Berenger-Jalla Alfred, ch. Pré-Fleuri 4.T. 26 ..."
3,Pré-Fleuri,Bussy-Du,NaN,NaN,['ingénieur'],chemindu Pré-Fleuri 8,8,"Bussy-Du Pasquier Ernest, ingénieur,Dir des Fo..."
4,Pré-Fleuri,Capt,Philippe,NaN,[],ch. de Pré-Fleuri 4,4,"— Philippe, ch. de Pré-Fleuri 4."
...,...,...,...,...,...,...,...,...
6727,Brillancourt,Pagani,Emile,NaN,['chauffeur'],ch. de Bril-lancourt 2,2,"Pagani Emile, chauffeur, ch. de Bril-lancourt 2."
6728,Brillancourt,Pagani,Lucie,NaN,[],ch. de Brillancourt 4,4,"— Lucie, ch. de Brillancourt 4."
6729,Brillancourt,Rahm-Malherbe,Victor,NaN,['compt.'],chem.Brillancourl 4,4,"Rahm-Malherbe Victor, compt., chem.Brillancour..."
6730,Brillancourt,Rod-von Gunten,Mathilde,NaN,['Vve'],ch. deBrillancourt 4,4,"Rod-von Gunten Mathilde, Vve, ch. deBrillancou..."


In [1]:
JOB_CAT_ABRV = {
                'téléphoniste': ['téléphone'],
                'chef': ['chef'],
                'policier': ['ag. de olice', 'police', 'polic'],
                'électricien': ['électric', 'élec'],
                'architecte': ['arch'],
                'professeur': ['prof', "prof d'école"],
                'couturier': ['cout'],
                'tapissier': ['tapiss', 'tap'],
                'agriculteur': ['agri', 'agricult'],
                'directeur': ['d.', 'dir'],
                'ménagère': ['ménag', 'men', 'mén'],
                'serrurier': ['serrur', 'serrar'],
                'commerçant': ['comm', 'commerc', 'commercant'],
                'maçon': ['macon'],
                'évangéliste': ['évang', 'évan'],
                'comptable': ['compt'],
                'veuve': ['vve', 'veuv'],
                'négociant': ['nég', 'négo'],
                'conducteur CFF': ['cond CFF', 'condu CFF', 'conduc CFF', 'conducteur CFF'],
                'étalagiste': ['étalag', 'étalag', 'étal'],
                'confiseur': ['confis', 'confisr'],
                'employé': ['emp', 'empi', 'emplo', 'empl', 'employée', 'empi, bur.'],
                'portier CFF': ['port CFF', 'portier CFF'],
                'portier': ['porte'],
                'voyagiste': ['voyagiste'],
                'gendarme': ['de gend', 'gendarm', 'gendarmerie', 'gend', 'sergent de gendarmerie', 'cap. de gend', 'capitaine gendarmerie'],
                'chauffeur': ['chauff', 'chauff'],
                'chauffeur CFF': ['chauff CFF', 'chauff CFF'],
                'céramiste': ['céram', 'cérami'],
                'brigadier de sûreté': ['brig', 'briga', 'brig de sûreté'],
                'assureur': ['assur', 'assuran', 'assureu'],
                'mécanicien': ['mécan', 'méca', 'mécano'],
                'mécanicien CFF': ['mécan CFF', 'méca CFF', 'mécano CFF'],
                'manœuvre': ['man', 'man', 'manœuv', 'manoeuvre'],
                'modiste': ['modis', 'mod'],
                'télégraphiste': ['télég', 'télégraphi', 'télégraphe'],
                'instituteur': ['instit', 'insituteu', 'insti', 'institutrice'],
                'représentant': ['repré', 'repr', 'représent'],
                'employé tramway': ['employédetrams', 'empl de tram', 'empl de trams'],
                "consul d'Italie": ["consul ditalie"],
                "vice-consul d'Italie": ["vice-consul ditalie"],
                'banquier': ['banq'],
                'dentiste': ['denti', 'dent'],
                'tabagiste': ['tabac', 'tabacs'],
                'étudiant': ['étud', 'étudia'],
                'tonnelier': ['tonnel'],
                'encaisseur': ['encaiss'],
                'cordonnier': ['cordonn', 'cordon', 'cordo'],
                'gérant': ['géran'],
                'ouvrier': ['ouvr'],
                'coiffeur': ['coiff', 'coiffeuse'],
                'tailleur': ['taill', 'tailleuse'],
                "inspecteur d'assurances": ["inspecteur d'assur"],
                'inspecteur CFF': ['inspect.CFF', 'inspect. CFF'],
                'rentier': ['rentière', 'renti', 'rent'],
                'tanneur': ['tann'],
                'vigneron': ['vign', 'vinger'],
                'commis': ['commi'],
                'commis CFF': ['com CFF', 'commis C.F.F.', 'com C.F.F.'],
                'pharmacien': ['pharmaa'],
                'couvreur': ['couvr'],
                'cuisinier': ['cuisin'],
                'restaurateur': ['restaur', 'g de rest'],
                'peintre': ['peint'],
                'horloger': ['horl', 'horloge'],
                'laitier': ['g laitier', 'laiterie'],
                'sommelier': ['sommel'],
                'violoncelliste': ['violoncel'],
                'sellier': ['sell'],
                'boulanger': ['boulan', 'boul'],
                'corsetière': ['corsetier'],
                'dactylographe': ['dactyl', 'dactylo'],
                'secrétaire': ['secrét'],
                'secrétaire CFF': ['secrét CFF'],
                'employé CFF': ['empi CFF', 'employ CFF'],
                'employé de commerce': ['emplo de commerce', 'empi de commerce'],
                'héliographe': ['hélio'],
                'médecin': ['méd', 'médec'],
                'géomètre': ['géo', 'géom'],
                'typographe': ['typo'],
                'menuisier': ['menu'],
                'consul de Belgique': ['consu de belgique'],
                'ingénieur': ['ingé', 'ing'],
                'domestique': ['domest'],
                'mouleur': ['moul'],
                'nettoyeur': ['netto', 'nett'],
                'artiste': ['art'],
                'publiciste': ['publi', 'publicité'],
                'concierge': ['concier', 'concierg'],
                'ramoneur': ['ramone'],
                'maréchal-ferrant': ['maréchal'],
                'pianiste': ['pian'],
                'chimiste': ['chim', 'chimi'],
                'livreur': ['livr', 'livraison'],
                'avocat': ['avoc'],
                'charcutier': ['charcut'],
                'épicier': ['épic'],
                'cafetier': ['cafet'],
                'lingère': ['ling', 'linger', 'linge'],
                'blanchisseuse': ['blanchis', 'blanchisseur', 'blanchiss'],
                'ébéniste': ['ében'],
                'fourreur': ['four', 'fourrures'],
                'relieur': ['reli', 'relieuse'],
                'teinturier': ['teint', 'tentur', 'teintures', 'teintureries'],
                'drogiste': ['drog', 'droguerie'],
                'commis postal': ['com. postal', 'comm. postal', 'commispostal'],
                'caissière': ['caisse', 'caissier'],
                'compositeur': ['compo', 'composit'],
                'charron': ['charr'],
                "agent d’affaires": ["agent d'aff"],
                'jounaliste': ['journal', 'journ'],
                'brodeuse': ['brod', 'brodeur'],
                'vendeur': ['vend', 'vendeuse'],
                'emballeur': ['emball', 'emballages'],
                'éditeur': ['éditeuse', 'édit', 'édition'],
                'jardinier': ['jardin', 'jardi'],
                'employé du gaz': ['empi du gaz', 'gaz'],
                'sténographe': ['sténo', 'steno'],
                "conseiller d'état": ["conseiller d'etat"],
                'attaché': ['attach', 'attachée'],
                'courtier': ['court', 'courtage'],
                'coutelier': ['couteaux', 'coutellerie'],
                'cocher': ['coch'],
                'employé des douanes': ['douanes', 'empi douanes', 'garde-front'],
                'administrateur': ['admin'],
                'fleuriste': ['fleurs', 'fleur'],
                'industriel': ['indust', 'indus'],
                'curé': ['cure', 'cur'],
                'imprimeur': ['impri', 'imprimerie'],
                'orfèvre': ['orfè', 'orf'],
                'oculiste': ['ocul'],
                'bijoutier': ['bijou', 'bijoux', 'bijoutière'],
                "chef d’institution": ["chef d'institut"],
                'papetier': ['papier', 'papeterie'],
                'religieux': ['religieuse'],
                'fonctionnaire': ['fonct', 'fonct postal', 'fonctionnaire postal'],
                'primeurs': ['prim', 'primeur'],
                'combustibles': ['combust'],
                'femme de chambre': ['f. de ch.', 'f de ch', 'f. de ch', 'f. de chambre'],
                'bottier': ['bottes', 'botte', 'bottière'],
                'sous-chef de gare': ['ss-chef de gare', 's.-chef de gare'],
                'employé postal': ['empi postal', 'employ postal', 'empl postal', 'employé de poste', 'employé des postes', 'emp. post'],
                'employé de banque': ['empi,de banque', 'emp. de banque'],
                'd. de mag': ['d de mag'],
                'professeur de piano et de chant': ['prof, depiano et chant'],
                'professeur de piano': ['prof, depiano', 'prof. de piano'],
                'professeur de musique': ['prof, musique'],
                'directeur du Crédit Suisse': ['dir Crédit Suisse'],
                'négociant en vins': ['négo. en vins', 'négoc. en vins'],
                'chef bureau CFF': ['chef bur. CFF'],
                'chef de train CFF': ['chef detrain CFF'],
                'masseur': ['masseuse', 'massouse'],
                'pension': ['pensio'],
                'ingénieur-professeur': ['ing. prof', 'ingé. prof'],
                'gouvernante': ['gouv'],
                'boucher': ['bouch'],
                'manœuvre CFF': ['man CFF'],
                'substitut du procureur génénéral': ['substitut duProcur. gén'],
                'facteur': ['fact'],
                'gypsier': ['gyps', 'gypse'],
                'faïences': ['faiences', 'faïence'],
                'commissionnaire': ['commissionnair'],
                'technicien': ['techni', 'techn'],
                'secrétaire à la direction de police': ['secrét. à ladir. de police'],
                'garde-malade': ['g.-mal.', 'g.-malade'],
                'employé de pharmacie': ['empi de phrmacie', 'employée de pharmacie', 'emp. de phar', 'emp. de pharmacie'],
                'tréfileur': ['tréfil', 'tréfileuse', 'tréfilage'],
                'professeur de peinture': ['prof. de peinture'],
                'directeur de pensionnat': ['dir. de pensionnat'],
                'droguiste': ['drog.', 'drogue', 'droguerie', 'droguerie en gros'],
                'vitrier': ['vitre', 'vit.', 'vitrerie'],
                'vulcanisateur': ['vulcan', 'vulcanisation', 'vulcanisatrice'],
                'chef de bureau postal': ['chef de bureaupostal'],
                'sculpteur': ['sculpt', 'sculpteuse', 'sculpterie'],
                'bouclier': ['boucle', 'boucl.'],
                'appareilleur': ['appreill'],
                'voyageur': ['voyageuse', 'voyag.'],
                'contrôleur': ['controleur', 'contrôl', 'contrôle', 'controle'],
                'plieur': ['plieuse', 'pli', 'pliage'],
                'charretier': ['charrette', 'charretière'],
                'inspecteur des bâtiments': ['inspect. des bâtiments'],
                'musicien': ['musique', 'musicien'],
                'batelier': ['batelière'],
                'ouvrier CFF': ['ouvrière CFF', 'ouvr. CFF'],
                'postillon': ['postill.'],
                'professeur de dessin': ['prof. dessin'],
                'briquetier': ['briques'],
                'pierriste': ['pierres', 'pierre'],
                'magasinier': ['magas.', 'magasin'],
                'charpentier': ['charpente'],
                'commissionnaire': ['commission'],
                'dessinateur': ['dessin'],
                'brasseur': ['brasseuse'],
                'électricien CFF': ['électr. CFF'],
                'constructeur de bateaux': ['const, de bateaux'],
                'machiniste': ['machine'],
                'lithographe': ['lithogr.', 'lith'],
                'monteur': ['monteuse', 'montage'],
                'opticien': ['optique', 'opticienne'],
                "huissier du Conseil d’Etat": ["huissier duConseil d’Etat"],
                'désinfecteur': ['désinfection', 'désinfect'],
                'pâtissier': ['pâtiss'],
                'infirmier': ['infirmière', 'infirm'],
                'pasteur': ['past'],
                'liftier': ['lift'],
                'employé de bureau': ['empl de bureau', 'empi bureau', 'employée de bureau'],
                "employé au consulat d'Italie": [],
                'consul du Pérou': ['Consul du Pérou'],
                'employé consulat': ['empi consulat']
}

for cat_abrv in JOB_CAT_ABRV.keys():
    new_list = JOB_CAT_ABRV[cat_abrv]
    new_list.append(cat_abrv)
    JOB_CAT_ABRV[cat_abrv] = new_list

JOB_CAT_FULL = JOB_CAT_ABRV.keys()

In [2]:
print("all_job = [")
for e in JOB_CAT_ABRV.keys():
    print(" " + '"'+ e + '"' + ",")

print("]")

all_job = [
 "téléphoniste",
 "chef",
 "policier",
 "électricien",
 "architecte",
 "professeur",
 "couturier",
 "tapissier",
 "agriculteur",
 "directeur",
 "ménagère",
 "serrurier",
 "commerçant",
 "maçon",
 "évangéliste",
 "comptable",
 "veuve",
 "négociant",
 "conducteur CFF",
 "étalagiste",
 "confiseur",
 "employé",
 "portier CFF",
 "portier",
 "voyagiste",
 "gendarme",
 "chauffeur",
 "chauffeur CFF",
 "céramiste",
 "brigadier de sûreté",
 "assureur",
 "mécanicien",
 "mécanicien CFF",
 "manœuvre",
 "modiste",
 "télégraphiste",
 "instituteur",
 "représentant",
 "employé tramway",
 "consul d'Italie",
 "vice-consul d'Italie",
 "banquier",
 "dentiste",
 "tabagiste",
 "étudiant",
 "tonnelier",
 "encaisseur",
 "cordonnier",
 "gérant",
 "ouvrier",
 "coiffeur",
 "tailleur",
 "inspecteur d'assurances",
 "inspecteur CFF",
 "rentier",
 "tanneur",
 "vigneron",
 "commis",
 "commis CFF",
 "pharmacien",
 "couvreur",
 "cuisinier",
 "restaurateur",
 "peintre",
 "horloger",
 "laitier",
 "sommelier",


In [3]:
len(JOB_CAT_ABRV)

208

In [4]:
def clean_job(job):
    job = job.replace(".", "")
    job = job.replace(",", "")
    job = job.replace("[", "")
    job = job.replace("]", "")
    job = job.replace('"', "")

    return job

In [5]:
final_job_list = list()
job_cat_values = [x for xs in list(JOB_CAT_ABRV.values()) for x in xs]

for index, row in df_index_street_final_best.iterrows():
    row_job = row['job']

    cleaned_job = clean_job(row_job)
    
    if len(cleaned_job) > 0:
        result = difflib.get_close_matches(cleaned_job, job_cat_values, n=1)
        print(result)
        if result:
            print(f"have {cleaned_job}, got {result[0]}")
            for key, list_job in JOB_CAT_ABRV.items():
                if result[0] in list_job:
                    print(f"then category {key}")
                    final_job_list.append(key)
        else:
            final_job_list.append(None)
    else:
        final_job_list.append(None)


NameError: name 'df_index_street_final_best' is not defined

In [ ]:
df_index_street_final_best.insert(1, 'cat_job', final_job_list)
df_index_street_final_best

,cat_loc,cat_job,lastname,firstname,org,job,loc,street_num,line
0,Pré-Fleuri,None,Abbet,Stéphanie,NaN,[],ch. de Pré-Fleuri 6,6,"— Stéphanie, ch. de Pré-Fleuri 6."
1,Pré-Fleuri,ingénieur,Amandmz-Dnpertnis,Victor,NaN,['ingénieurOFF'],ch. Pré-Fleuri 2,2,"Amandmz-Dnpertnis Victor, ingénieurOFF., ch. P..."
2,Pré-Fleuri,None,Berenger-Jalla,Alfred,NaN,[],ch. Pré-Fleuri 4.T.,4,"Berenger-Jalla Alfred, ch. Pré-Fleuri 4.T. 26 ..."
3,Pré-Fleuri,ingénieur,Bussy-Du,NaN,NaN,['ingénieur'],chemindu Pré-Fleuri 8,8,"Bussy-Du Pasquier Ernest, ingénieur,Dir des Fo..."
4,Pré-Fleuri,None,Capt,Philippe,NaN,[],ch. de Pré-Fleuri 4,4,"— Philippe, ch. de Pré-Fleuri 4."
...,...,...,...,...,...,...,...,...,...
6727,Brillancourt,chauffeur,Pagani,Emile,NaN,['chauffeur'],ch. de Bril-lancourt 2,2,"Pagani Emile, chauffeur, ch. de Bril-lancourt 2."
6728,Brillancourt,None,Pagani,Lucie,NaN,[],ch. de Brillancourt 4,4,"— Lucie, ch. de Brillancourt 4."
6729,Brillancourt,comptable,Rahm-Malherbe,Victor,NaN,['compt.'],chem.Brillancourl 4,4,"Rahm-Malherbe Victor, compt., chem.Brillancour..."
6730,Brillancourt,None,Rod-von Gunten,Mathilde,NaN,['Vve'],ch. deBrillancourt 4,4,"Rod-von Gunten Mathilde, Vve, ch. deBrillancou..."


In [ ]:
print(df_index_street_final_best['cat_job'].count())

# we have more than half with a job


4316


In [ ]:
df_index_street_final_best.to_csv('1951_index_street_cat_jobs.csv', index=False)

In [ ]:
df_num = pd.read_csv("1951_index_street.csv")

In [ ]:
df_num

,cat_loc,lastname,firstname,org,job,loc,line
0,Pré-Fleuri,Abbet,Stéphanie,NaN,[],ch. de Pré-Fleuri 6,"— Stéphanie, ch. de Pré-Fleuri 6."
1,Pré-Fleuri,Amandmz-Dnpertnis,Victor,NaN,['ingénieurOFF'],ch. Pré-Fleuri 2,"Amandmz-Dnpertnis Victor, ingénieurOFF., ch. P..."
2,Pré-Fleuri,Berenger-Jalla,Alfred,NaN,[],ch. Pré-Fleuri 4.T.,"Berenger-Jalla Alfred, ch. Pré-Fleuri 4.T. 26 ..."
3,Pré-Fleuri,Bussy-Du,NaN,NaN,['ingénieur'],chemindu Pré-Fleuri 8,"Bussy-Du Pasquier Ernest, ingénieur,Dir des Fo..."
4,Pré-Fleuri,Capt,Philippe,NaN,[],ch. de Pré-Fleuri 4,"— Philippe, ch. de Pré-Fleuri 4."
...,...,...,...,...,...,...,...
6727,Brillancourt,Pagani,Emile,NaN,['chauffeur'],ch. de Bril-lancourt 2,"Pagani Emile, chauffeur, ch. de Bril-lancourt 2."
6728,Brillancourt,Pagani,Lucie,NaN,[],ch. de Brillancourt 4,"— Lucie, ch. de Brillancourt 4."
6729,Brillancourt,Rahm-Malherbe,Victor,NaN,['compt.'],chem.Brillancourl 4,"Rahm-Malherbe Victor, compt., chem.Brillancour..."
6730,Brillancourt,Rod-von Gunten,Mathilde,NaN,['Vve'],ch. deBrillancourt 4,"Rod-von Gunten Mathilde, Vve, ch. deBrillancou..."


In [ ]:
street = df_num.loc[~((df_num['cat_loc'] == 'Mon-loisir') | (df_num['line'].str.contains('montchoisi')))]
#street = df_num.loc[~((df_num['cat_loc'] == 'Mon-loisir') | (df_num['cat_loc'].str.contains('Mont-Choisi')))]

In [ ]:
street

,cat_loc,lastname,firstname,org,job,loc,line
0,Pré-Fleuri,Abbet,Stéphanie,NaN,[],ch. de Pré-Fleuri 6,"— Stéphanie, ch. de Pré-Fleuri 6."
1,Pré-Fleuri,Amandmz-Dnpertnis,Victor,NaN,['ingénieurOFF'],ch. Pré-Fleuri 2,"Amandmz-Dnpertnis Victor, ingénieurOFF., ch. P..."
2,Pré-Fleuri,Berenger-Jalla,Alfred,NaN,[],ch. Pré-Fleuri 4.T.,"Berenger-Jalla Alfred, ch. Pré-Fleuri 4.T. 26 ..."
3,Pré-Fleuri,Bussy-Du,NaN,NaN,['ingénieur'],chemindu Pré-Fleuri 8,"Bussy-Du Pasquier Ernest, ingénieur,Dir des Fo..."
4,Pré-Fleuri,Capt,Philippe,NaN,[],ch. de Pré-Fleuri 4,"— Philippe, ch. de Pré-Fleuri 4."
...,...,...,...,...,...,...,...
6727,Brillancourt,Pagani,Emile,NaN,['chauffeur'],ch. de Bril-lancourt 2,"Pagani Emile, chauffeur, ch. de Bril-lancourt 2."
6728,Brillancourt,Pagani,Lucie,NaN,[],ch. de Brillancourt 4,"— Lucie, ch. de Brillancourt 4."
6729,Brillancourt,Rahm-Malherbe,Victor,NaN,['compt.'],chem.Brillancourl 4,"Rahm-Malherbe Victor, compt., chem.Brillancour..."
6730,Brillancourt,Rod-von Gunten,Mathilde,NaN,['Vve'],ch. deBrillancourt 4,"Rod-von Gunten Mathilde, Vve, ch. deBrillancou..."


In [ ]:
df_super = pd.read_csv("")